# **ConformalNL2LTL: Translating Natural Language Instructions into Temporal Logic Formulas with Conformal Correctness Guarantees**

***Jun Wang<sup>1&#42;</sup>, David Smith Sundarsingh<sup>1&#42;</sup>, Jyotirmoy V. Deshmukh, Yiannis Kantaros<sup>1</sup>***

*Department of Electrical and Systems Engineering, Washington University in St Louis<sup>1</sup>*

*Department of Computer Science, University of Southern California<sup>2</sup>*

&#42; indicates equal contribution.


Copyright 2025 [Kantaros Lab @ WashU](https://sites.wustl.edu/kantaroslab/). All rights reserved.

Please check project webpage [conformalnl2ltl.github.io](https://conformalnl2ltl.github.io/) for more information.



In [ ]:
API_KEY_PRI = "OpenAI API key of your primary model"
API_KEY_SEC = "OpenAI API key of your secondary model"

## LLM Configurations

***Google drive access will be requested to save data***

In [ ]:
!pip install openai==1.55.2 httpx==0.27.2 --quiet
import ollama
import json
import numpy as np
from itertools import combinations
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from openai import OpenAI
import os:
    os.makedirs(path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00


In [ ]:
#@title API Call Function
client = OpenAI(api_key=API_KEY_PRI, base_url="https://api.deepseek.com")
client_gpt = OpenAI(api_key=API_KEY_PRI)
def LLM_DS(prompt):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": prompt}],
        stream=False
    )
    text = response.choices[0].message.content
    index = text.find('\n')
    first_line = text[:index] if index != -1 else text

    return response, first_line

def GPT(messages):
    response = client_gpt.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        temperature=1,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={"type": "text"}
    )
    return response, response.choices[0].message.content

def formulator(role, text):
    message = {
    "role": role,
    "content": [{
            "text": text,
            "type": "text"
    }]}
    return message

def LLM(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,
        top_p=1,
        temperature=1
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(decoded)

    # Split at <start_of_turn>model and take the last part
    lines = decoded.splitlines()

    response = None
    found_model = False

    for line in lines:
        if found_model:
            if line.strip():         # first non-empty line after "model"
                response = line.strip()
                break
        elif line.strip() == "model":
            found_model = True

    if response is None or "model" in response:
        print(f"Nod model output found.")
        return ""
    else:
        print(f"The output is: {response}")
        return response

In [ ]:
#@title Semantic similarity computation
def extract_text_and_number(text):
    # Extract the number
    number_match = re.search(r'\d+', text)
    number = int(number_match.group()) if number_match else None
    # Extract the non-numeric part
    text_part = re.sub(r'\d+', '', text).strip('_')
    text_part = text_part.replace('_', ' ')
    return text_part, number

# text-embedding-ada-002, text-embedding-3-large, text-embedding-3-small
def get_embedding(text, model="text-embedding-3-large"):
    response = ollama.embeddings(model='nomic-embed-text', prompt=text)
    return response['embedding']

def cos(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def semantic_similarity(text1, text2):
    w1, n1 = extract_text_and_number(text1)
    w2, n2 = extract_text_and_number(text2)
    vector1 = np.array(get_embedding(w1))
    vector2 = np.array(get_embedding(w2))
    if n1 != n2:
      # stop considering semantic similarity if the numbers do not match
      return 0
    cos_sim = cos(vector1, vector2)
    similarity = cos_sim
    return similarity

print(semantic_similarity('stay_hall_2', 'hall_2e'))

In [ ]:
def check_validity(new_string):
  pattern = r'(?<!\w)(<>|!|->|&&|\|\||\[\]|U|\(|\)|/|[a-zA-Z0-9_]+)(?!\w)'
  found_keys = re.findall(pattern, new_string)
  if len(found_keys) == 1:
    return True, found_keys
  elif len(found_keys) > 1:
        return False, found_keys  # Invalid string with the conflicting keys
  else:
    return False, []
  
class CustomOutput:
    def __init__(self, file):
        self.file = file

    def write(self, message):
        print(message, end='')  # Print to console
        self.file.write(message)  # Write to file

    def flush(self):
        pass  # For compatibility with `print`

## System Prompt

The translation of Natural Language (NL) task into Linear Temporal Logic (LTL) formula is constructed incrementally as a sequence of interdependent multiple-choice-question-answering (MCQA) tasks.

At each time step, we query the LLM $m$ times to generate $m$ candidate response, each response can be either
*   An operator comes from a fixed pre-defined set
*   An atomic proposition that is generated based on the rules defined in the prompt

Operators we considered:

|     Symbol    |  Explanation  |
|:-------------:|:-------------:|
|      $<>$     |   Eventually  |
|      $!$      |    Negation   |
| $\rightarrow$ |  Implication  |
|     $\&\&$    |      And      |
|     $||$    |       OR      |
|      $[]$     |     Always    |
|      $U$      |     Until     |
|      $($      |  Left Bracket |
|      $)$      | Right Bracket |
|      $/$      | Ending Symbol |


In [ ]:
#@title List of Temporal/Logical Operators
operators = {
    '<>': 'eventually',
    '!' : 'negation',
    '->': 'implication',
    '&&': 'and',
    '||': 'or',
    '[]': 'always',
    'U' : 'until',
    '(' : 'left bracket',
    ')' : 'right bracket',
    '/': 'only upon LTL formula completion.'
}

opkeys = list(operators.keys())

In [ ]:
#@title Prompt for the primary model
formatted_operators = '\n'.join([ops[0] + ' # ' + ops[1] for ops in operators.items()])
gpt_system_prompt = f"""You are a helpful assistant tasked with translating natural language (NL) instructions into Linear Temporal Logic (LTL) formulas step by step.

At each step, generate exactly 1 element either an operator strictly from the set of Logical and Temporal Operators listed below or an Atomic Predicates by following the construction rules below.

Possible Options:
For temporal and logical operators, you must pick only one from the below elements and nothing else:
<> # eventually
! # negation
-> # implication
&& # and
|| # or
[] # always
U # until
( # left bracket
) # right bracket
/ # only upon LTL formula completed

Atomic Predicates construction rules (AP):
The robot can a) go to a location; b) pick up an item; c) put down an item; and d) take a photo.
At each step, if you need to generate an atomic predicate for one of the capabilities, you must follow the rules below:
Go to [location X] is written as location_X
Pick up [item X] is written as p_item_X
Put down is written as pd
take a photo is written as photo

!! You can only select one operator or one AP at a given step !!

Do not generate "<>(" at the same step but generate "<>" first and then "(".

Example NL Instruction: Deliver package 4 to store 5 after picking it up from warehouse 1 as you keep visiting charger 4.
Process Example:
[User]:
Formula so far:
Step 1:
[Assistant]:
<>

[User]:
Formula so far: <>
Step 2:
[Assistant]:
(

[User]:
Formula so far: <>(
Step 3:
[Assistant]:
warehouse_4

[User]:
Formula so far: <>(warehouse_1
Step 4:
[Assistant]:
&&

[User]:
Formula so far: <>(warehouse_1&&
Step 5:
[Assistant]:
<>

[User]:
Formula so far: <>(warehouse_1&&<>
Step 6:
[Assistant]:
(

[User]:
Formula so far: <>(warehouse_1&&<>(
Step 7:
[Assistant]:
p_package_4

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4
Step 8:
[Assistant]:
&&

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&
Step 9:
[Assistant]:
<>

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>
Step 10:
[Assistant]:
(

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(
Step 11:
[Assistant]:
store_5

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(store_5
Step 12:
[Assistant]:
&&

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(store_5&&
Step 13:
[Assistant]:
pd

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(store_5&&pd
Step 14:
[Assistant]:
)

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(store_5&&pd)
Step 15:
[Assistant]:
)

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(store_5&&pd))
Step 16:
[Assistant]:
)

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(store_5&&pd)))
Step 17:
[Assistant]:
&&

[User]:
Formula so far:<>(warehouse_1&&<>(p_package_4&&<>(store_5&&pd)))&&
Step 18:
[Assistant]:
[]

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(store_5&&pd)))&&[]
Step 19:
[Assistant]:
<>

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(store_5&&pd)))&&[]<>
Step 20:
[Assistant]:
charger_4

[User]:
Formula so far: <>(warehouse_1&&<>(p_package_4&&<>(store_5&&pd)))&&[]<>charger_4
Step 21:
[Assistant]:
/

Final LTL formula:
<>(warehouse_1&&<>(p_package_4&&<>(store_5&&pd)))&&[]<>charger_4/

Some other examples of NL instructions and their corresponding LTL formulas:

NL Instruction: Take block 3 from house 3 to house 4.
Final LTL formula:
<>(house_3&&<>(p_block_3&&<>(house_4&&pd)))/

NL Instruction: Keep taking a picture of house 6.

Process Example:
[User]:
Formula so far:
Step 1:
[Assistant]:
[]

[User]:
Formula so far: []
Step 2:
[Assistant]:
<>

[User]:
Formula so far: []<>
Step 3:
[Assistant]:
(

[User]:
Formula so far: []<>(
Step 4:
[Assistant]:
house_6

[User]:
Formula so far: []<>(house_6
Step 5:
[Assistant]:
&&

[User]:
Formula so far: []<>(house_6&&
Step 6:
[Assistant]:
photo

[User]:
Formula so far: []<>(house_6&&photo
Step 7:
[Assistant]:
)

[User]:
Formula so far: []<>(house_6&&photo)
Step 8:
[Assistant]:
/

Final LTL formula:
[]<>(house_6&&photo)/

NL Instruction: Stay in interstate 64 until you reach gas station 4 as you take box 1 from house 5 to then house 6.

Process Example:
[User]:
Formula so far:
Step 1:
[Assistant]:
insterstate_64

[User]:
Formula so far: insterstate_64
Step 2:
[Assistant]:
U

[User]:
Formula so far: insterstate_64U
Step 3:
[Assistant]:
gas_station_4

[User]:
Formula so far: insterstate_64Ugas_station_4
Step 4:
[Assistant]:
&&

[User]:
Formula so far: insterstate_64Ugas_station_4&&
Step 5:
[Assistant]:
<>

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>
Step 6:
[Assistant]:
(

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(
Step 7:
[Assistant]:
house_5

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5
Step 8:
[Assistant]:
&&

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&
Step 9:
[Assistant]:
<>

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>
Step 10:
[Assistant]:
(

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(
Step 11:
[Assistant]:
p_box_1

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1
Step 12:
[Assistant]:
&&

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&
Step 13:
[Assistant]:
<>

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&<>
Step 14:
[Assistant]:
(

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&<>(
Step 15:
[Assistant]:
house_6

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&<>(house_6
Step 16:
[Assistant]:
&&

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&<>(house_6&&
Step 17:
[Assistant]:
pd

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&<>(house_6&&pd
Step 18:
[Assistant]:
)

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&<>(house_6&&pd)
Step 19:
[Assistant]:
)

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&<>(house_6&&pd))
Step 20:
[Assistant]:
)

[User]:
Formula so far: insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&<>(house_6&&pd)))
Step 21:
[Assistant]:
/

Final LTL formula:
insterstate_64Ugas_station_4&&<>(house_5&&<>(p_box_1&&<>(house_6&&pd)))/

NL Instruction: Do not enter lot 3 until you reach building 2 while going to building 2 and taking a picture.
Final LTL formula:
!lot_3Ubuilding_2&&<>(building_2&&photo)/

NL Instruction: While never going to street 5, take a picture of mall 4.

Process Example:
[User]:
Formula so far:
Step 1:
[Assistant]:
[]

[User]:
Formula so far: []
Step 2:
[Assistant]:
!

[User]:
Formula so far: []!
Step 3:
[Assistant]:
street_5

[User]:
Formula so far: []!street_5
Step 4:
[Assistant]:
&&

[User]:
Formula so far: []!street_5&&
Step 5:
[Assistant]:
<>

[User]:
Formula so far: []!street_5&&<>
Step 6:
[Assistant]:
(

[User]:
Formula so far: []!street_5&&<>(
Step 7:
[Assistant]:
mall_4

[User]:
Formula so far: []!street_5&&<>(mall_4
Step 8:
[Assistant]:
&&

[User]:
Formula so far: []!street_5&&<>(mall_4&&
Step 9:
[Assistant]:
photo

[User]:
Formula so far: []!street_5&&<>(mall_4&&photo
Step 10:
[Assistant]:
)

[User]:
Formula so far: []!street_5&&<>(mall_4&&photo)
Step 11:
[Assistant]:
/

Final LTL formula:
[]!street_5&&<>(mall_4&&photo)/

NL Instruction: Go to street 3 and stay there, but do not go there till you reach intersection 4
Final LTL formula:
<>[]street_3&&!street_3Uintersection_4/

NL Instruction: Take a picture of statue 2 and then go to bench 5
Final LTL formula:
<>(statue_2&&photo&&<>bench_5)/

NL Instruction: Stay in statue 2 as you go to bench 5
Final LTL formula:
[]statue_2&&<>bench_5/

NL Instruction: Stay in area 4 until you take letter 2 from office 4 to office 8
Final LTL formula:
area_4U(office_4&&<>(p_letter_2&&<>(office_8&&pd)))/

Task Instructions You Must Follow:
1. For the given NL instruction, build the LTL formula step by step.
2. When you are asked to select an element:
2a. You must only choose exactly one element, either a single operator listed or an AP generated, that will build up the LTL formula to satisfy the NL description
2b. '[]' and '!' are to be considered separate elements and should not be generated together.
2c. '<>' and '(' are to be considered separate elements and should not be generated together.
2d. '!' and '(' are to be considered separate elements and should not be generated together.
2e. Do not generate part of the element
2f. The operator 'U' is not preceeded or succeeded by operators
3. When you generate AP as an option, make sure the name of the location or item in your response has the same semantic meaning as described in the NL instruction.
4. Whenever you generate a left bracket "(", you must generate a corresponding right bracket ")" later in the formula so that all the brackets are complete and maintain proper precedence.
5. If the NL task indicates that the AP generated is the first of a sequence, the AP must be preceeded by '('.
"""

print(system_description_prompt)

## Calibration

We generated a dataset of 1,000 task-formula pairs that are of three levels of complexity.

|        | # of Atomic Propositions | Dataset Size |
|:------:|:------------------------:|:------------:|
|  Easy  |          {1, 2}          |      365     |
| Medium |          {3, 4}          |      440     |
| Hard   |        More than 4       |      195     |




### Load the Calibration and Validation Datasets

In [ ]:
#@title Select Coverage Level
coverage_in_percentage = 95 # @param {type:"number"}
alpha = 1-coverage_in_percentage/100

In [ ]:
import numpy as np
import random 



valid_root_path = f'Input the path where the results have to be stored'
if not os.path.exists(valid_root_path):
    os.makedirs(valid_root_path)

group_index = str(f'Validation_{len(os.listdir(valid_root_path))+1}')
valid_path = os.path.join(valid_root_path, str(group_index))
if not os.path.exists(valid_path):
    os.makedirs(valid_path)


total_sample = 220
calib_count = 200

# obtain the quantile for both models
ensemble_score = "/Dataset/max_scores.json"
with open(ensemble_score, 'r') as score_file:
   ensemblescores = json.load(score_file)
n = len(ensemblescores)
calib_idxs = random.sample(range(n), calib_count)
compute = []
for i in calib_idxs:
   compute.append(int(ensemblescores[i]['score']))
for score in ensemblescores:
   compute.append(score['score'])
quantile_ensemble = np.quantile(compute, 1 - alpha)

print(f"The scores are {compute}")

test_json_record = []
test_nl_tasks = []
for i in test_idxs:
   test_nl_tasks.append(scores[i]['nltask'])
ds_dataset = 'final_calib.json'
with open(ds_dataset, 'r') as dataset_file:
    total_dataset = json.load(dataset_file)
task_map = {item["nlTask"]: item["ltlequ"] for item in total_dataset}
# test_data =  {k: task_map[k] for k in test_nl_tasks if k in task_map}
test_data  = {}
for k in test_nl_tasks:
    if k in task_map:
      test_data[k] = task_map[k]

print(f"Now setting up validation group {group_index}\nData will be saved at: {valid_path}")
print(f"Calibration Size: {calib_count} | Validation Size: {total_sample-calib_count}")
print(f"Ensemble quantile for {coverage_in_percentage}% coverage: {quantile_ensemble}")
  

with open(os.path.join(valid_path, 'test_data.json'), 'w') as f:
    json.dump(test_data, f, indent=4)

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(

JSON file loaded successfully!

Length of Dataset: 1000

Example Data: {'nlTask': 'Take a picture of building 2', 'ltlequ': '<>(building_2&&photo)/'}


## Validation with Help

After you obtain the calibration dataset, we can now sample a subset from it to compute the quantile based on custom $1-\alpha$ coverage level

In [ ]:
def CallCPT(api_call_per_step, messages, quantile_gpt, deepseek_predset, mywriter):
    results = {}
    gpt_response = []
    for t in range(1, api_call_per_step+1):
        mywriter.write(f"\nGPT Running in Progress: {t}/{api_call_per_step}")
        response, response_text = GPT(messages)
        valid, found_keys = check_validity(response_text)
        gpt_response.append({'is_valid': valid, 'text': response_text, 'found_keys': found_keys})
        mywriter.write(f"\n-------[GPT][Valid: {valid}]---------\n" + f"LLM Response {t}: " + response_text.replace('\n', '') + f"\nKeys Found: {found_keys}")
        if valid:
            if found_keys[0] not in results:
                results[found_keys[0]] = 0
            results[found_keys[0]] += 1
        if len(found_keys) == 0:
            continue
        print(f"\nAdded to {found_keys[0]}")
    print(f"The results vector is {results}")
    total = sum(results.values())
    valid_results = {key: value / total for key, value in results.items()}

    for k, v in valid_results.items():
      print(f"{k}: {v}\n")
    
    # List to keep track of keys to remove
    keys_to_remove = []
    for key1, key2 in combinations(results.keys(), 2):
      if semantic_similarity(key1, key2) > 0.7:
        print(f"High similarity {semantic_similarity(key1, key2)} between '{key1}' and '{key2}'")
        if valid_results[key1] >= valid_results[key2]:
            valid_results[key1] += valid_results[key2]
            keys_to_remove.add(key2)
        else:
            valid_results[key2] += valid_results[key1]
            keys_to_remove.add(key1)

    # Remove marked keys from the dictionary
    for key in keys_to_remove:
        if key in valid_results:
            del valid_results[key]
    
    pred_set = []
    for k, v in valid_results.items():
      if v >= 1-quantile_gpt:
        pred_set.append(k)

    intersection = list(set(pred_set) & set(deepseek_predset))
    mywriter.write(f"\n===\nDeepSeek Pred Set: {deepseek_predset}\nGPT Pred Set: {pred_set}\nIntersection Prediction Set of DeepSeek and GPT is: {intersection}\n===\n")
    return intersection, pred_set

In [ ]:
with open(os.path.join(valid_path, 'test_data.json'), "r") as f:
    test_data = json.load(f)

alpha = 0.01
opkeys = operators.keys()
api_per_step = 10
gpt_api_call_per_step = 10

for i, (natural_language_task, ltlequ) in enumerate(test_data.items()):
    ltlequ = ltlequ[0]
    print(natural_language_task, ltlequ)
    words = re.findall(r'[a-zA-TV-Z_0-9]+|<>|&&|\|\||\[\]|\(|\)|U|[^\s\w]', ltlequ)
    
    
    foldername = os.path.join(valid_path, str(f"Valid_{i+1}"))
    if not os.path.exists(foldername):
        os.makedirs(foldername)

    data = []
    data_loc = os.path.join(foldername, 'data.npy')
    print(f"Data will be saved in {data_loc}")

    scores = []
    text_name = "system_description"
    text_path = f'/Users/davidsmithsundarsingh/Language Models/ConformalNL2LTL/{text_name}.txt'
    with open(text_path, 'r') as f:
        content = f.read()
    formula_so_far = ''
    prompt_ds = content

    # Setup GPT basic prompt 
    system_message = formulator(role='user', text=gpt_system_prompt)
    messages = [system_message]
    messages.append(formulator(role='user', text=natural_language_task))

    step_count = 1
    help_rate = 0
    while True:
        print(f"Currently at step {step_count}")
        user_input = f"Formula so far: {formula_so_far}\nStep {step_count}:"
        messages.append(formulator(role='user', text=user_input))
        note = os.path.join(foldername, str(f"step_{step_count}.txt"))
        f = open(note, 'w')
        my = CustomOutput(f)
        my.write(f"\nTask:\nNL instruction: {natural_language_task}\nFormula_so_far: {formula_so_far}\nStep: {step_count}")
        prompt_ds += f"\nTask:\nNL instruction: {natural_language_task}\nFormula so far: {formula_so_far}\nStep {step_count}:\nGENERATE ONLY ONE SYMBOL\n"
        results = {}
        # Obtain M LLM responses from the options
        responses = []
        deepseek_responses = []
        for t in range(1,api_per_step+1):
            my.write(f"\n\n=============================\n")
            my.write(f"API Call {t}/{api_per_step} | Formula so far: {formula_so_far} | GT:{ltlequ}")
            response, response_text = GPT(messages)
            valid, found_keys = check_validity(response_text)
            responses.append({'is_valid': valid, 'text': response_text, 'found_keys': found_keys})
            my.write(f"\n-------[ChatGPT][Valid: {valid}]-------\n" + f"LLM Response {t}: " + response_text.replace('\n', '') + f"\nKeys Found: {found_keys}")

            if valid:
                if found_keys[0] not in results:
                    results[found_keys[0]] = 0
                results[found_keys[0]] += 1

        print(f"\nThe results vector is {results}")

        if not results:
            print("!!!!!!!!!!!!!!!!!!!!!!!!!Nothing in the results!!!!!!!!!!!!!!!!!!!!!!!!!")

        # Aggregate similar APs together
        keys_to_remove = set()
        for key1, key2 in combinations(results.keys(), 2):
            if semantic_similarity(key1, key2) > 0.8:
                print(f"High similarity {semantic_similarity(key1, key2)} between {key1} and {key2}")
                results[key1] += results[key2]
                keys_to_remove.add(key2)
        
        for key in keys_to_remove:
            del results[key]
        
        # Compute valid probabilities
        total = sum(results.values())
        valid_results = {key: value / total for key, value in results.items()}
        # my.write(f"True element is {words[step_count-1]}\n")

        pred_set = []
        for k, v in valid_results.items():
            if v >= 1 - alpha: # check the pred set using deepseek first
                pred_set.append(k)
        
        if len(pred_set) != 1:
            results_deepseek = {}
            deepseek_response = []
            for t in range(1, api_per_step+1):
                my.write(f"\nDeepseek Running in Progress: {t}/{api_per_step}")
                response, response_text = LLM_DS(prompt_ds)
                valid, found_keys = check_validity(response_text)
                deepseek_response.append({'is_valid': valid, 'text': response_text, 'found_keys': found_keys})
                my.write(f"\n-------[Deepseek][Valid: {valid}]---------\n" + f"LLM Response {t}: " + response_text.replace('\n', '') + f"\nKeys Found: {found_keys}")
                if valid:
                    if found_keys[0] not in results_deepseek:
                        results_deepseek[found_keys[0]] = 0
                    results_deepseek[found_keys[0]] += 1
                if len(found_keys) == 0:
                    continue
                print(f"\nAdded to {found_keys[0]}")
            print(f"The results vector is {results_deepseek}")
            total = sum(results_deepseek.values())
            valid_results = {key: value / total for key, value in results_deepseek.items()}

            for k, v in valid_results.items():
                print(f"{k}: {v}\n")
            
            # List to keep track of keys to remove
            keys_to_remove = []
            for key1, key2 in combinations(results_deepseek.keys(), 2):
                if semantic_similarity(key1, key2) > 0.8:
                    print(f"High similarity {semantic_similarity(key1, key2)} between '{key1}' and '{key2}'")
                    if valid_results[key1] >= valid_results[key2]:
                        valid_results[key1] += valid_results[key2]
                        keys_to_remove.append(key2)
                    else:
                        valid_results[key2] += valid_results[key1]
                        keys_to_remove.append(key1)

            # Remove marked keys from the dictionary
            for key in keys_to_remove:
                if key in valid_results:
                    del valid_results[key]
            
            pred_set_deepseek = []
            for k, v in valid_results.items():
                if v >= 1-alpha:
                    pred_set_deepseek.append(k)

            intersection = list(set(pred_set_deepseek) & set(pred_set))
            my.write(f"\n===\nDeepSeek Pred Set: {pred_set_deepseek}\nGPT Pred Set: {pred_set}\nIntersection Prediction Set of DeepSeek and GPT is: {intersection}\n===\n")

            if len(intersection) == 1:
                # DeepSeek + GPT results in a singleton
                pred_set[0] = intersection[0]
            else:

                if len(intersection) == 0:
                    print(f"The translation has failed")
                    break

                # Otherwise, go to ask for help from user 
                user_input_msg = f"I  need help!\nNL: {natural_language_task}\nGT is: {ltlequ}\nThe current formula is {formula_so_far}\nChoose an option of the following:"
                my.write(f"\n{user_input_msg}")
                for i, option in enumerate(pred_set):
                    my.write(f"\n{i+1}: {option}")
                user_input = input(user_input_msg)
                user_input = int(user_input)
                my.write(f"\n\nUser Selected: {pred_set[int(user_input)-1]}")
                pred_set[0] = pred_set[int(user_input)-1]
                help_rate += 1
        if len(pred_set) == 0:
            print(f"Translation has failed!")
            break
        
        formula_so_far += pred_set[0]

        prompt_ds += f"{pred_set[0]}\n"
        messages.append(formulator(role='assistant', text=pred_set[0]))
        step_count += 1
        my.write(f"\nNew Formula: {formula_so_far}\n")

        if formula_so_far[-1] == '/':
            my.write(f"The current formula is now ended with /, exiting...")
            break
    print(f"\n{natural_language_task}")
    print(f"The helps asked: {help_rate} and step count : {step_count-1}")
    correct = True
    if ltlequ != formula_so_far:
        useranswer = input(f"Formula so far:{formula_so_far}\nEnter 'no' if you think it is wrong: ")
        if useranswer == 'no':
            correct = False 
    print(f"The final formula is {'correct' if correct else 'wrong'}")
    json_filename = 'result.json'
    if os.path.isfile(json_filename):
        with open(json_filename, 'r') as f:
            jsondata = json.load(f)
    else:
        jsondata = []
    new_dict = {
        "nlTask": natural_language_task,
        "ltlequ": ltlequ,
        "api_per_step": api_per_step,
        'help': help_rate,
        "correct": correct,
        'total_step': step_count -1
    }
    jsondata.append(new_dict)
    with open(json_filename, 'w') as f:
      json.dump(jsondata, f, indent=4) # Use indent for pretty printing

    print(f"New data length: {len(jsondata)}")

    # compute help_rate
    help = 0
    total = 0
    correct = 0
    for data in jsondata:
        help = help + data['help']
        total = total + data['total_step']
        if data['correct']:
            correct = correct + 1
    print(f"\n\n\nHelp Rate: {help/total}\nAccuracy: {correct/len(jsondata)}")